## Create Dataset to be Used for MVP Predictions

In [1]:
import pandas as pd
from helper_functions import conference_mapping

### Read CSVs into Dataframes and Select Relevant Columns

In [2]:
# Read CSVs into Dataframes
player_season_averages = pd.read_csv("data/Player Per Game.csv")
award_voting = pd.read_csv("data/Player Award Shares.csv")
team_season_info = pd.read_csv("data/Team Summaries.csv")
per_100_poss = pd.read_csv("data/Per 100 Poss.csv")
player_season_info = pd.read_csv("data/Player Season Info.csv")
advanced_stats = pd.read_csv("data/Advanced.csv")

# Filter relevant columns
relevant_season_averages = player_season_averages[["seas_id", "season", "player_id", "lg", "player", "g", "fg_percent", "fta_per_game", "orb_per_game", "drb_per_game", "ast_per_game", "stl_per_game", "blk_per_game", "tov_per_game", "pts_per_game"]]
relevant_award_voting = award_voting[["seas_id", "player_id", "share", "award", "winner", "season"]]
relevant_team_info = team_season_info[["abbreviation", "team", "season", "w", "l"]]
relevant_per_100_poss = per_100_poss[["seas_id", "player_id", "pts_per_100_poss"]]
relevant_advanced_stats = advanced_stats[["seas_id", "player_id", "per", "ts_percent", "vorp", "ws", "bpm"]]
relevant_player_season_info = player_season_info[["seas_id", "player_id", "tm", "season"]]

### Generate Season Standings

In [3]:
# Filter out pre Merger Data
relevant_team_info = relevant_team_info[relevant_team_info['season'] >= 1976]

# Map conference to team
relevant_team_info['conference'] = relevant_team_info['team'].map(conference_mapping)

# Calculate winning percentage
relevant_team_info['winning_percentage'] = relevant_team_info['w'] / (relevant_team_info['w'] + relevant_team_info['l'])

# Rank teams by conference
relevant_team_info['conference_rank'] = relevant_team_info.groupby(['season', 'conference'])['winning_percentage'].rank(method='dense', ascending=False)

# Sort by season, conference, and conference rank
relevant_team_info.sort_values(by=['season', 'conference', 'conference_rank'], inplace=True)
relevant_team_info.reset_index(drop=True, inplace=True)
print(relevant_team_info.head(5))

  abbreviation                 team  season     w     l conference  \
0          BOS       Boston Celtics    1976  54.0  28.0    Eastern   
1          NYA        New York Nets    1976  55.0  29.0    Eastern   
2          CLE  Cleveland Cavaliers    1976  49.0  33.0    Eastern   
3          WSB   Washington Bullets    1976  48.0  34.0    Eastern   
4          BUF       Buffalo Braves    1976  46.0  36.0    Eastern   

   winning_percentage  conference_rank  
0            0.658537              1.0  
1            0.654762              2.0  
2            0.597561              3.0  
3            0.585366              4.0  
4            0.560976              5.0  


### Associate Player with Team

In [4]:
relevant_team_info_copy = relevant_team_info.copy()

relevant_team_info_copy['number_of_games_in_season'] = (relevant_team_info_copy['w'] + relevant_team_info_copy['l'])

# Merge relevant_player_season_info with relevant_team_info
player_season_record = pd.merge(relevant_player_season_info,
                                relevant_team_info_copy,
                                left_on=['tm', 'season'],
                                right_on=['abbreviation', 'season'],
                                how='left')

# Drop unnecessary columns post-merge
player_season_record.drop(['abbreviation', 'w', 'l', 'season', "team"], axis=1, inplace=True)

### Create a Cumulative MVP Count Column and Combine Dataframes

In [15]:
# Filter for Relevant Award
relevant_award_voting = relevant_award_voting[relevant_award_voting['award'] == 'nba mvp']

# Sort
relevant_award_voting = relevant_award_voting.sort_values(by=['player_id', 'season'])

# Calculate previous mvps
relevant_award_voting['winner'] = relevant_award_voting['winner'].astype(int)

relevant_award_voting['previous_mvps'] = relevant_award_voting.groupby('player_id')['winner'].cumsum().shift(fill_value=0)

train_test_data = relevant_season_averages.copy()

# Drop data that is not from NBA
train_test_data = train_test_data[train_test_data['lg'] == "NBA"]

# Drop the 'lg' column from the DataFrame
train_test_data.drop('lg', axis=1, inplace=True)

# Get relevant award voting data
relevant_award_voting.rename(columns={'share': 'mvp_vote_share'}, inplace=True)

# Merge the dataframes
train_test_data = pd.merge(train_test_data, relevant_award_voting[['seas_id', 'player_id', 'mvp_vote_share', 'previous_mvps']], 
                         on=["seas_id", "player_id"], 
                         how="left")

train_test_data['mvp_vote_share'].fillna(0, inplace=True)
train_test_data['previous_mvps'].fillna(0, inplace=True)

train_test_data = pd.merge(train_test_data, relevant_per_100_poss, 
                         on=["seas_id", "player_id"], 
                         how="left")

train_test_data = pd.merge(train_test_data, relevant_advanced_stats, 
                         on=["seas_id", "player_id"], 
                         how="left")

train_test_data = pd.merge(train_test_data, player_season_record,  
                         on=["seas_id", "player_id"], 
                         how="left")

train_test_data["percent_games_played"] = train_test_data["g"] / train_test_data["number_of_games_in_season"]

train_test_data['mvp_race_rank'] = train_test_data.groupby(['season'])['mvp_vote_share'].rank(method='dense', ascending=False)

# Drop unnecessary columns post-merge
train_test_data.drop(['number_of_games_in_season', 'g', 'conference'], axis=1, inplace=True)

# Filter data from before merger
data_for_prediction = train_test_data.copy()[train_test_data['season'] == 2024]

train_test_data = train_test_data[train_test_data['season'] >= 1976]
train_test_data = train_test_data[train_test_data['season'] <= 2023]

### Clean Data and Write to CSV

In [17]:
# Assuming df is your DataFrame
columns_to_exclude = ['player', 'tm']
dfs = [train_test_data, data_for_prediction]

for df in dfs:
    for column in df.columns:
        if column not in columns_to_exclude:
            # Remove non-numeric characters (keep digits and decimal points)
            train_test_data[column] = train_test_data[column].astype(str).str.replace('[^\d.]', '', regex=True)
            # Convert to numeric, force non-convertible values to NaN (or use errors='coerce')
            train_test_data[column] = pd.to_numeric(train_test_data[column], errors='coerce')

print("Shape of train test data before dropping bad data: \n" + str(train_test_data.shape))

print("Shape of prediction data before dropping bad data: \n" + str(data_for_prediction.shape))

# This effectively drops all players who were traded mid season since they don't have a winning percentage, team seed etc...
# This is no issue because no MVP has ever been traded mid season
train_test_data.dropna(inplace=True)
data_for_prediction.dropna(inplace=True)
print("Shape of train test data after dropping bad data: " + str(train_test_data.shape))
print("Shape of prediction data after dropping bad data: " + str(data_for_prediction.shape))

train_test_data.to_csv("train_test_data.csv", index=False)
data_for_prediction.to_csv("data_for_prediction.csv", index=False)

Shape of train test data before dropping bad data: 
(21578, 26)
Shape of prediction data before dropping bad data: 
(602, 26)
Shape of train test data after dropping bad data: (21578, 26)
Shape of prediction data after dropping bad data: (602, 26)


### Sanity Check

In [18]:
# Assuming df is your DataFrame
top_10_mvp_votes = train_test_data.sort_values(by='mvp_vote_share', ascending=False).head(10)

print(top_10_mvp_votes[['player', 'season', 'mvp_vote_share', "previous_mvps", "mvp_race_rank"]])

                 player  season  mvp_vote_share  previous_mvps  mvp_race_rank
6006      Stephen Curry    2016           1.000            1.0            1.0
15033  Shaquille O'Neal    2000           0.998            0.0            1.0
7696       LeBron James    2013           0.998            3.0            1.0
12806     Kevin Garnett    2004           0.991            0.0            1.0
17101    Michael Jordan    1996           0.986            3.0            1.0
7066       Kevin Durant    2014           0.986            0.0            1.0
21469        Larry Bird    1986           0.981            2.0            1.0
9424       LeBron James    2010           0.980            1.0            1.0
21833        Larry Bird    1985           0.978            1.0            1.0
8629       Derrick Rose    2011           0.977            0.0            1.0
